In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/17 01:17:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/17 01:17:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/17 01:17:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark.version

'3.3.2'

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-17 01:17:40--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.36.218, 13.226.36.130, 13.226.36.196, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.36.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  62.4MB/s    in 1.0s    

2025-03-17 01:17:41 (62.4 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [16]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

df = df.repartition(4)

df.write.mode("overwrite").parquet('data/pq/yellow/2024/10/')

In [17]:
!ls -lh data/pq/yellow/2024/10/

total 97M
-rw-r--r-- 1 Night Night   0 Mar 17 01:26 _SUCCESS
-rw-r--r-- 1 Night Night 25M Mar 17 01:26 part-00000-d9858aca-7a17-4c17-93fc-4f20ebe4ef69-c000.snappy.parquet
-rw-r--r-- 1 Night Night 25M Mar 17 01:26 part-00001-d9858aca-7a17-4c17-93fc-4f20ebe4ef69-c000.snappy.parquet
-rw-r--r-- 1 Night Night 25M Mar 17 01:26 part-00002-d9858aca-7a17-4c17-93fc-4f20ebe4ef69-c000.snappy.parquet
-rw-r--r-- 1 Night Night 25M Mar 17 01:26 part-00003-d9858aca-7a17-4c17-93fc-4f20ebe4ef69-c000.snappy.parquet


In [18]:
df = spark.read.parquet('data/pq/yellow/2024/10/')

In [19]:
df.createOrReplaceTempView('yellow_2024_10')

In [13]:
spark.sql("""select * from yellow_2024_10 limit 5""").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-10 10:34:57|  2024-10-10 10:53:48|              1|          3.4|         1|                 N|         140|          74|           1|       20.5|  0.0|    0.5|      0.2

In [20]:
spark.sql("""
SELECT
    COUNT(*)
FROM 
    yellow_2024_10
WHERE
    to_date(tpep_pickup_datetime) = '2024-10-15';
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [28]:
spark.sql("""
SELECT
    max(unix_timestamp(tpep_dropoff_datetime ) - unix_timestamp(tpep_pickup_datetime)) / 60 / 60 diffInHours
FROM 
    yellow_2024_10
;
""").show()

+-----------------+
|      diffInHours|
+-----------------+
|162.6177777777778|
+-----------------+



In [29]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-17 01:33:15--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.36.196, 13.226.36.130, 13.226.36.218, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.36.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-17 01:33:15 (89.2 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [30]:
zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

zones.createOrReplaceTempView('taxi_zone_lookup')

In [32]:
spark.sql("""select * from taxi_zone_lookup limit 5""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [33]:
spark.sql("""
SELECT taxi_zone_lookup.Zone, count(*) trip_count
from taxi_zone_lookup 
join yellow_2024_10
on taxi_zone_lookup.LocationID = yellow_2024_10.PULocationID
group by 1
order by 2
limit 1
""").show()

+--------------------+----------+
|                Zone|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
+--------------------+----------+

